In [126]:
def read_data(path):
    f = open(path,"r")
    return [line.strip() for line in f.readlines()]

In [130]:
demo_dataset = read_data("demo.txt")
demo_dataset2 = read_data("demo2.txt")
full_dataset = read_data("data.txt")
demo_dataset

['[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]',
 '[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]',
 '[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]',
 '[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]',
 '[7,[5,[[3,8],[1,4]]]]',
 '[[2,[2,2]],[8,[8,1]]]',
 '[2,9]',
 '[1,[[[9,3],9],[[9,0],[0,7]]]]',
 '[[[5,[7,4]],7],1]',
 '[[[[4,2],2],6],[8,7]]']

In [10]:
from typing import List, Tuple
Number = Tuple[List[int], List[int]]

# Part 1

In [11]:
def parse(string: str) -> Number:
    depth = 0
    numbers = []
    depths = []
    for char in string:
        if char == "[":
            depth += 1
        elif char == "]":
            depth -= 1
        elif char != ",":
            n = int(char)
            numbers.append(n)
            depths.append(depth)
    return (numbers, depths)

In [14]:
assert parse("[[1,2],[[3,4],5]]") == ([1, 2, 3, 4, 5], [2, 2, 3, 3, 2])

In [29]:
def explode(numbers: List[int], depths: List[int], i: int) -> Number:
    if i > 0:
        numbers[i-1] += numbers[i]
    if i + 2 < len(numbers):
        numbers[i + 2] += numbers[i + 1]
    new_numbers = [ *(numbers[:i] if i > 0 else []), 0, *numbers[i + 2:]]
    new_depths = [ *(depths[:i] if i > 0 else []), depths[i] - 1, *depths[i + 2:]]
    return (new_numbers, new_depths)

In [33]:
assert explode(*parse("[[[[[9,8],1],2],3],4]"), 0) == parse("[[[[0,9],2],3],4]")
assert explode(*parse("[7,[6,[5,[4,[3,2]]]]]"), 4) == parse("[7,[6,[5,[7,0]]]]")
assert explode(*parse("[[6,[5,[4,[3,2]]]],1]"), 3) == parse("[[6,[5,[7,0]]],3]")
assert explode(*parse("[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]"), 3) == parse("[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]")
assert explode(*parse("[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]"), 7) == parse("[[3,[2,[8,0]]],[9,[5,[7,0]]]]")

In [87]:
def split(numbers: List[int], depths: List[int], i: int) -> Number:
    left = numbers[i] // 2
    right = (numbers[i] + 1) // 2
    new_numbers = [ *(numbers[:i] if i > 0 else []), left, right, *numbers[i + 1:]]
    new_depths = [ *(depths[:i] if i > 0 else []), depths[i] + 1, depths[i] + 1, *depths[i+1:]]
    return (new_numbers, new_depths)

In [88]:
assert split([0, 7, 4, 15, 0, 13, 1, 1], [4, 4, 3, 3, 4, 4, 2, 2], 3) == ([0, 7, 4, 7, 8, 0, 13, 1, 1], [4, 4, 3, 4, 4, 4, 4, 2, 2])

In [90]:
def reduce_step(numbers: List[int], depths: List[int]) -> Number:
    for i in range(len(depths)):
        if depths[i] > 4:
            return explode(numbers, depths, i)
    for i in range(len(numbers)):
        if numbers[i] > 9:
            return split(numbers, depths, i)

In [93]:
def reduce(number: Number) -> Number:
    current = number
    while (not all([i<=4 for i in current[1]])) or (not all([i < 10 for i in current[0]])):
        current = reduce_step(*current)
    return current

In [94]:
assert reduce(parse("[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]")) == parse("[[[[0,7],4],[[7,8],[6,0]]],[8,1]]")

In [99]:
def add(number1: Number, number2: Number) -> Number:
    numbers = [*number1[0], *number2[0]]
    depths = [ *[i + 1 for i in number1[1]] , *[j + 1 for j in number2[1]]]
    return (numbers, depths)

In [101]:
assert reduce(add(parse("[[[[4,3],4],4],[7,[[8,4],9]]]"), parse("[1,1]"))) == parse("[[[[0,7],4],[[7,8],[6,0]]],[8,1]]")

In [180]:
def magnitude(number: Number) -> int:
    depth = number[1]
    numbers = number[0]
    max_depth = max(depth)
    new_depths = []
    new_numbers = []
    i = 0
    while i < len(numbers):
        if depth[i] != max_depth:
            new_numbers.append(numbers[i])
            new_depths.append(depth[i])
            i += 1
        else:
            new_numbers.append(3 * numbers[i] + 2 * numbers[i + 1])
            new_depths.append(depth[i] - 1)
            i += 2
    if len(new_numbers) == 1:
        return new_numbers[0]
    else:
        return magnitude((new_numbers, new_depths))

In [181]:
magnitude(([0, 7, 8, 8, 0, 6, 9, 6, 6, 6, 7, 0, 8, 9], [4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 4]))

2478

In [182]:
assert magnitude(parse("[[1,2],[[3,4],5]]")) == 143
assert magnitude(parse("[[[[0,7],4],[[7,8],[6,0]]],[8,1]]")) == 1384
assert magnitude(parse("[[[[1,1],[2,2]],[3,3]],[4,4]]")) == 445
assert magnitude(parse("[[[[3,0],[5,3]],[4,4]],[5,5]]")) == 791
assert magnitude(parse("[[[[5,0],[7,4]],[5,5]],[6,6]]")) == 1137
assert magnitude(parse("[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]")) == 3488

In [183]:
def compute(rows: List[str]) -> int:
    parsed = [ parse(row) for row in rows]
    current = parsed[0]
    for i in range(1, len(parsed)):
        current = reduce(add(current, parsed[i]))
    return magnitude(current)

In [184]:
assert compute(demo_dataset) == 3488

In [185]:
assert compute(demo_dataset2) == 4140

In [186]:
compute(full_dataset)

3734

# Part 2

In [187]:
from copy import copy

In [190]:
def max_sum(dataset):
    parsed = [ parse(row) for row in dataset]
    current_max = 0
    for row1 in parsed:
        for row2 in parsed:
            if row1 != row2:
                result = reduce(add(copy(row1), copy(row2)))
                value = magnitude(result)
                current_max = max(current_max, value)
    return current_max

In [192]:
assert max_sum(demo_dataset2) == 3993

In [193]:
max_sum(full_dataset)

4837